In [1]:
#Using Spotipy, a lightweight Python library for the Spotify Web API. Guide at link below
#https://spotipy.readthedocs.io/en/latest/#
import spotipy, sys, csv
import pprint as pp
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

#Load Application ID and API Key for Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id='0de0796ce8d6430d9896525ccb8c845d',
                                                      client_secret='4b1c63bc8de34e7a9cc75b105f3c3875')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
#Read in csv of Singles and convert to DataFrame
top100singles = pd.read_csv('top100singles.csv')
list_of_dicts = []

#Loop Through all Singles in top100 singles csv
for index,row in top100singles.iterrows():
    
    title_and_artist = row['Single'] + ', ' + row['Artist']

    #Make API Call using search method, querying for "Title, Artist"
    #Limit to just the top response
    results = spotify.search(q=title_and_artist, type='track',limit=1)

    #temporary dictionary to store all of the track details
    temp = {}
    temp['name'] = results['tracks']['items'][0]['name']
    temp['popularity'] = results['tracks']['items'][0]['popularity']

    #if there are multiple artist on album, store as one value separated by commas
    multi_Artist = False
    for j in range(len(results['tracks']['items'][0]['artists'])):
        if multi_Artist == False:
            temp['artists'] = results['tracks']['items'][0]['artists'][j]['name']
            multi_Artist = True
        else:
            temp['artists'] = temp['artists'] + ', ' + results['tracks']['items'][0]['artists'][j]['name']

    list_of_dicts.append(temp)

In [10]:
#convert list of dictionaries to DataFrame, make index start at 1 instead of 0
df = pd.DataFrame(list_of_dicts)
df.columns = ['Artist(s)', 'Title (Spotify)','Popularity']
df.head()

,Artist(s),Title (Spotify),Popularity
0,"Bing Crosby, Ken Darby Singers, John Scott Tro...",White Christmas,71
1,Elton John,Something About The Way You Look Tonight - Edi...,60
2,Mungo Jerry,In the Summertime,67
3,Bing Crosby,Silent Night,46
4,Bill Haley & His Comets,(We're Gonna) Rock Around The Clock,61


In [12]:
#concatenate results, and start index at 1
wiki_and_spotify = pd.concat([top100singles,df],axis=1)
wiki_and_spotify.index += 1
wiki_and_spotify.head()

,Artist,Single,Release Year,Sales (in millions),Artist(s),Title (Spotify),Popularity
1,Bing Crosby,White Christmas,1942,50,"Bing Crosby, Ken Darby Singers, John Scott Tro...",White Christmas,71
2,Elton John,Candle in the Wind 1997 / Something About the ...,1997,33,Elton John,Something About The Way You Look Tonight - Edi...,60
3,Mungo Jerry,In the Summertime,1970,30,Mungo Jerry,In the Summertime,67
4,Bing Crosby,Silent Night,1935,30,Bing Crosby,Silent Night,46
5,Bill Haley & His Comets,Rock Around the Clock,1954,25,Bill Haley & His Comets,(We're Gonna) Rock Around The Clock,61


In [14]:
wiki_and_spotify.to_csv('output.csv',index=False)